In [1]:
%load_ext autoreload
%autoreload 2
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTransitions
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.model_based_agent import ModelBasedAgent
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import torch.nn as nn
import torch
import gym
from cs285.infrastructure import pytorch_util as ptu
from torchdiffeq import odeint
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax

In [2]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
device = "cuda" if torch.cuda.is_available() else "cpu"
ptu.init_gpu(use_gpu=True if device=="cuda" else False)

gpu
Using GPU id 0


In [3]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
num_layers = 4
hidden_size = 128
lr=0.001
agent_key, key = jax.random.split(key)
def make_dynamic_model(ob_dim: int, ac_dim: int):
    return ptu.build_mlp(
        input_size=ob_dim+ac_dim,
        output_size=ob_dim,
        n_layers=num_layers,
        size=hidden_size
    )
def make_optimizer(parameters):
    return torch.optim.Adam(parameters, lr=lr)

mb_agent = ModelBasedAgent(
    env=env,
    make_dynamics_model=make_dynamic_model,
    make_optimizer=make_optimizer,
    ensemble_size=10,
    mpc_horizon=100,
    mpc_strategy="cem",
    mpc_discount=1.0,
    mpc_num_action_sequences=1000,
    mpc_dt=0.05,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
    mode="vanilla"
)
replay_buffer = ReplayBufferTransitions()
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=10, max_length=200, key=key)
for traj in trajs:
    replay_buffer.batched_insert(
    observations=traj["observation"],
    actions=traj["action"],
    rewards=traj["reward"],
    next_observations=traj["next_observation"],
    dones=traj["done"],
    dts=traj["dt"]
)

100%|██████████| 10/10 [00:00<00:00, 30.84it/s]


In [4]:
mb_agent.update_statistics(
    obs=replay_buffer.observations,
    acs=replay_buffer.actions,
    next_obs = replay_buffer.next_observations
)

In [5]:
for n in trange(10):
    for i in range(mb_agent.ensemble_size):
        batch = replay_buffer.sample(64)
        mb_agent.update(i, batch["observations"], batch["actions"], batch["next_observations"], batch["dts"])

100%|██████████| 10/10 [00:00<00:00, 13.50it/s]


In [10]:
trajs, _ = sample_n_trajectories(
    env=env,
    policy=mb_agent,
    ntraj=10,
    max_length=200,
    key=key
)

 10%|█         | 1/10 [01:55<17:22, 115.84s/it]

In [7]:
ob = env.reset()
ac = mb_agent.get_action(ob, key=None)

In [8]:
ac

array([0.30612447])